to aquire af lep b nircam coron data:

1st epoch: jwst_download.py -v --instrument nircam --propID 4558 -f uncal.fits --date_select 2023-10-11 2023-10-14

2nd epoch: jwst_download.py -v --instrument nircam --propID 4558 -f uncal.fits --date_select 2023-11-01 2023-11-03

3rd epoch: jwst_download.py -v --instrument nircam --propID 4558 -f uncal.fits --date_select 2024-01-01+

In [1]:
from __future__ import division

# =============================================================================
# IMPORTS
# =============================================================================

import os
import pdb
import sys 

import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import numpy as np

from spaceKLIP import database, coron1pipeline, coron2pipeline, coron3pipeline, pyklippipeline, imagetools, analysistools

# plotting
import matplotlib.pyplot as plt
# plt.style.use('/Users/wbalmer/balmer.mplstyle')
# import seaborn as sb
# sb.set_context('talk')

In [32]:
# Set the input and output directories and grab the input FITS files.
reduced = True
cleanalign = False
aligned = False

blur = True
pad = True
coadd = False
crop = False

# epoch = '_e1'
epoch = '_e3'

data_root='DATA/JWST/NIRCam/04451/F210M/'

if aligned and coadd:
    input_dir = os.path.join(data_root, 'coadded/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    # input_dir = './spaceklip/aligned/'
    input_dir = os.path.join(data_root, 'aligned/')#f'./spaceklip/aligned{epoch}/'
    # input_dir = './spaceklip/aligned/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign:
    input_dir = os.path.join(data_root, 'recentered/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = os.path.join(data_root, 'stage2/')
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    # input_dir = './04558/'
    input_dir = os.path.join(data_root, 'uncal/') #f'/Users/wbalmer/data/jwst/aflep/uncal{epoch}/'
    # input_dir = '/Users/wbalmer/data/jwst/aflep/uncal_e3/'
    # input_dir = './uncal_e2/'
    # input_dir = './uncal_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

# additional PSF references dir
ref_ids = ['01411','04454']#['01563','03337','03840','03947','03973','03989','04050','04090','04558']#,'04014'# 
additionalrefs = []
for i in range(len(ref_ids)):
    input_dir_ref = 'DATA/JWST/NIRCam/%s/F210M/stage2/' % ref_ids[i]
    # read files in additional PSF references input dir
    additionalrefs = additionalrefs + sorted([input_dir_ref + f for f in os.listdir(input_dir_ref) if f.endswith('.fits')])
# list of PSF reference files in main input dir
#psffitsfiles = [f for f in fitsfiles if 'jw04558003001' in f]+[f for f in fitsfiles if 'jw04558006001' in f]+[f for f in fitsfiles if 'jw04558009001' in f]
psffitsfiles = [f for f in fitsfiles if '10001' in f]
# global list of ALL files
allpaths = fitsfiles + additionalrefs
# global list of all PSF reference files 
psfpaths = additionalrefs + psffitsfiles


output_dir = data_root

In [ ]:
#fitsfiles = fitsfiles[-4:]

In [ ]:
psffitsfiles

In [ ]:
additionalrefs

In [33]:
#psfpaths.pop(-10)
psfpaths

['DATA/JWST/NIRCam/01411/F210M/stage2/jw01411022001_06101_00001_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411023001_06101_00001_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411027001_03105_00001_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411027001_03105_00002_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411027001_03105_00003_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411027001_03105_00004_nrca2_calints.fits',
 'DATA/JWST/NIRCam/01411/F210M/stage2/jw01411027001_03105_00005_nrca2_calints.fits',
 'DATA/JWST/NIRCam/04454/F210M/stage2/jw04454001001_03106_00001_nrca2_calints.fits',
 'DATA/JWST/NIRCam/04454/F210M/stage2/jw04454002001_03106_00001_nrca2_calints.fits',
 'DATA/JWST/NIRCam/04454/F210M/stage2/jw04454002001_03106_00002_nrca2_calints.fits',
 'DATA/JWST/NIRCam/04454/F210M/stage2/jw04454002001_03106_00003_nrca2_calints.fits',
 'DATA/JWST/NIRCam/04454/F210M/stage2/jw04454002001_03106_00004_n

In [34]:
for file in psfpaths:
    data = pyfits.open(file)
    subarray_type = data[0].header['SUBARRAY']
    file_name = data[0].header['FILENAME'] 
    print(file_name, subarray_type)

jw01411022001_06101_00001_nrca2_calints.fits SUB640A210R
jw01411023001_06101_00001_nrca2_calints.fits SUB640A210R
jw01411027001_03105_00001_nrca2_calints.fits SUB640A210R
jw01411027001_03105_00002_nrca2_calints.fits SUB640A210R
jw01411027001_03105_00003_nrca2_calints.fits SUB640A210R
jw01411027001_03105_00004_nrca2_calints.fits SUB640A210R
jw01411027001_03105_00005_nrca2_calints.fits SUB640A210R
jw04454001001_03106_00001_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00001_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00002_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00003_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00004_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00005_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00006_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00007_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00008_nrca2_calints.fits SUB640A210R
jw04454002001_03106_00009_nrca2_calints.fits SUB640A210R
jw04451010001_03106_00001_nrca2

In [12]:
len(psfpaths)

32

In [38]:
for f in fitsfiles:
    if f in psfpaths:
        print(f)
        fitsfiles.remove(f)
fitsfiles

DATA/JWST/NIRCam/04451/F210M/stage2/jw04451010001_03106_00008_nrca2_calints.fits


['DATA/JWST/NIRCam/04451/F210M/stage2/jw04451009001_03106_00001_nrca2_calints.fits']

In [31]:
lvls = ['uncal','stage1','stage2']#,'medsub','bpcleaned','nanreplaced','blurred','padded','recentered','aligned']
for r in ref_ids:
    for l in lvls:
        if not os.path.exists('DATA/JWST/NIRCam/'+r+'/F210M/'+l):
            #print('DATA/JWST/NIRCam/%s/F200W/%s does not exist' % (r,l))
            os.mkdir('DATA/JWST/NIRCam/'+r+'/F210M/'+l)
            #print('Made DATA/JWST/NIRCam/%s/F200W/%s' % (r,l))
        #else:
            #print('DATA/JWST/NIRCam/%s/F200W/%s exists' % (r,l))
        for f in os.listdir('DATA/JWST/NIRCam/04451/F210M/%s' % l):
            if r in f:
                os.rename('DATA/JWST/NIRCam/04451/F210M/%s/%s' % (l,f),'DATA/JWST/NIRCam/%s/F210M/%s/%s' % (r,l,f))
        #('DATA/JWST/NIRCam/04558/F200W/'+l)

In [4]:
fitsfiles

['DATA/JWST/NIRCam/04451/F200W/uncal/jw04451009001_03108_00001_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00001_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00002_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00003_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00004_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00005_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00006_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00007_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00008_nrca2_uncal.fits',
 'DATA/JWST/NIRCam/04451/F200W/uncal/jw04451010001_0310a_00009_nrca2_uncal.fits']

In [39]:
# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=psfpaths,
                             bgpaths=None)

[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R
TYPE  EXP_TYPE DATAMODL TELESCOP ... BUNIT       ROLL_REF      BLURFWHM
---- --------- -------- -------- ... ------ ------------------ --------
 SCI NRC_CORON   STAGE2     JWST ... MJy/sr  78.64611073377338      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr  84.34374297947535      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr  94.34377257226282      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr  76.43515498091017      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr  76.43512964577464      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr   76.4351475030814      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr   76.4351276349287      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr  76.43510809588601      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr 118.13031400111635      nan
 REF NRC_CORON   STAGE2     JWST ... MJy/sr 11

In [40]:
Database.summarize()

NIRCAM_F210M_MASK210R
	STAGE2: 27 files;	1 SCI, 26 REF


In [35]:
Database.obs['JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R']

TYPE,EXP_TYPE,DATAMODL,TELESCOP,TARGPROP,TARG_RA,TARG_DEC,INSTRUME,DETECTOR,FILTER,CWAVEL,DWAVEL,PUPIL,CORONMSK,EXPSTART,NINTS,EFFINTTM,SUBARRAY,NUMDTHPT,XOFFSET,YOFFSET,APERNAME,PPS_APER,PIXSCALE,BUNIT,CRPIX1,CRPIX2,RA_REF,DEC_REF,ROLL_REF,BLURFWHM,FITSFILE,MASKFILE
object,object,object,object,object,float64,float64,object,object,object,float64,float64,object,object,float64,int64,float64,object,int64,float64,float64,object,object,float64,object,float64,float64,float64,float64,float64,float64,object,object
SCI,NRC_CORON,STAGE2,JWST,51-Eri,69.40084830923283,-2.473978657058863,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60361.04672686481,23,33.48672,SUB640A210R,1,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,69.40084064933855,-2.473975714823927,78.64611073377338,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw04451009001_03106_00001_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw04451009001_03106_00001_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,BETA-PIC,86.82124645396839,-51.06597537480891,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.0006975463,90,16.74336,SUB640A210R,1,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,86.8211392579084,-51.06595909746579,84.34374297947535,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411022001_06101_00001_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411022001_06101_00001_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,BETA-PIC,86.82124645440717,-51.065975369881095,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.07867079861,90,16.74336,SUB640A210R,1,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,86.82114645823793,-51.065947312936714,94.34377257226282,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411023001_06101_00001_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411023001_06101_00001_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,ALPHA-PIC,102.04682398822847,-61.93982240398298,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.46926974537,10,16.74336,SUB640A210R,5,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,102.04670168354227,-61.93957387933296,76.43515498091017,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00001_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00001_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,ALPHA-PIC,102.04682398793308,-61.93982240347202,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.47203493056,10,16.74336,SUB640A210R,5,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,102.04669512739714,-61.93957885386965,76.43512964577464,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00002_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00002_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,ALPHA-PIC,102.04682398763761,-61.93982240296093,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.47480085648,10,16.74336,SUB640A210R,5,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,102.0466909801027,-61.93957077744016,76.4351475030814,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00003_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00003_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,ALPHA-PIC,102.04682398734222,-61.93982240244998,NIRCAM,NRCA2,F210M,2.0982217923787,0.20553790735177,MASKRND,MASK210R,60021.477566030095,10,16.74336,SUB640A210R,5,0.0,0.0,NRCA2_MASK210R,NRCA2_MASK210R,0.030673395,MJy/sr,353.0,353.0,102.0467081548231,-61.939568808641674,76.4351276349287,nan,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00004_nrca2_calints.fits,DATA/JWST/NIRCam/04451/F210M/padded/jw01411027001_03105_00004_nrca2_calints_psfmask.fits
REF,NRC_CORON,STAGE2,JWST,ALPHA-PIC,102.04682398704684,-61.93982240193902,NIRCAM,NRCA2,F210M,2.0982

In [48]:
files = list(Database.obs['JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R']['FITSFILE'])
#files[0][:29]
for f in files:
    hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
    print(hdu.header['SUBARRAY'])

[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw04451009001_03106_00001_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings:WARNING] 
[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw01411022001_06101_00001_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings:WARNING] 
[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw01411023001_06101_00001_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings

SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R


[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw04451010001_03106_00001_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings:WARNING] 
[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw04451010001_03106_00002_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings:WARNING] 
[py.warnings:WARNING] /var/folders/0x/ms4g19v94ks3vl4stzvlgh44000184/T/ipykernel_1821/2857612725.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='DATA/JWST/NIRCam/04451/F210M/stage2/jw04451010001_03106_00003_nrca2_calints.fits'>
[py.warnings:WARNING]   hdu = pyfits.open(f[:29]+'stage2/'+f[-44:])[0]
[py.warnings

SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R
SUB640A210R


In [16]:
# select a subset of obs, usually we are only interested in F200W+F444W

select_obs = [
              # 'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
                'JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R',
              # 'JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_FULL'
              # 'JWST_NIRCAM_NRCALONG_F356W_MASKRND_MASK335R_SUB320A335R',
              # 'JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

In [ ]:
Database.obs['JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R']['TYPE']!='REF'

In [41]:
if not reduced:
    coron1pipeline.run_obs(database=Database,
                           steps={'saturation': {'n_pix_grow_sat': 1,
                                                 'grow_diagonal': False},
                                  'refpix': {'odd_even_columns': True,
                                             'odd_even_rows': True,
                                             'nlower': 4,
                                             'nupper': 4,
                                             'nleft': 4,
                                             'nright': 4,
                                             'nrow_off': 0,
                                             'ncol_off': 0},
                                  'dark_current': {'skip': True},
                                  'persistence': {'skip': True},
                                  'jump': {'rejection_threshold': 4.,
                                           'three_group_rejection_threshold': 4.,
                                           'four_group_rejection_threshold': 4.,
                                           'maximum_cores': 'all'},
                                  'ramp_fit': {'save_calibrated_ramp': False,
                                               'maximum_cores': 'all'}},
                            subdir='stage1')

if not reduced:
    coron2pipeline.run_obs(database=Database,
                           steps={'outlier_detection': {'skip': False}},
                           subdir='stage2')

ImageTools = imagetools.ImageTools(Database)

if not cleanalign:
    ImageTools.update_nircam_centers()

if not cleanalign:
    ImageTools.subtract_median(types=['SCI', 'SCI_TA', 'SCI_BG', 'REF', 'REF_TA', 'REF_BG'],
                                   subdir='medsub')

# Fix bad pixels using custom spaceKLIP routines. Multiple routines can be
# combined in a custom order by joining them with a + sign.
# - bpclean: use sigma clipping to find additional bad pixels.
# - custom: use custom map to find additional bad pixels.
# - timemed: replace pixels which are only bad in some frames with their
#            median value from the good frames.
# - dqmed:   replace bad pixels with the median of surrounding good
#            pixels.
# - medfilt: replace bad pixels with an image plane median filter.
if not cleanalign:
    ImageTools.fix_bad_pixels(method='bpclean+timemed+dqmed+medfilt',
                              bpclean_kwargs={'sigclip': 5,
                                              'shift_x': [-1, 0, 1],
                                              'shift_y': [-1, 0, 1]},
                              custom_kwargs={},
                              timemed_kwargs={},
                              dqmed_kwargs={'shift_x': [-1, 0, 1],
                                            'shift_y': [-1, 0, 1]},
                              medfilt_kwargs={'size': 4},
                              subdir='bpcleaned')

if not cleanalign:
    ImageTools.replace_nans(cval=0.,
                            types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                            subdir='nanreplaced')

if not cleanalign:
    if blur:
        ImageTools.blur_frames()

if not cleanalign:
    if crop:
        ImageTools.crop_frames(npix=10)
    
    if pad:
        ImageTools.pad_frames(
                              npix=[32, 33, 32, 33],
                              # npix=[1, 144, 73, 72], # shortwave
                              types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                              cval=0.
                             )
        

if not cleanalign:
    ImageTools.recenter_frames(
        spectral_type='F0V',
    )

if not aligned:
    ImageTools.align_frames(
        subdir='aligned', #f'aligned{epoch}',
        method='fourier',
        align_algo='leastsq',
    )

[spaceKLIP.imagetools:INFO] --> Concatenation JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: jw04451009001_03106_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: old = (321.80, 331.50), new = (321.80, 331.50)
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: jw01411022001_06101_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: old = (321.00, 330.90), new = (321.00, 330.90)
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: jw01411023001_06101_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: old = (321.00, 330.90), new = (321.00, 330.90)
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: jw01411027001_03105_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Update NIRCam coronagraphy centers: old = (321.00, 330.9

[spaceKLIP.imagetools:WARNING]   --> The following frames might not be properly aligned: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411022001_06101_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 0.00 mas


[spaceKLIP.imagetools:WARNING]   --> The following frames might not be properly aligned: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
[spaceKLIP.imagetools:WARNING]  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
[spaceKLIP.imagetools:WARNING]  48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
[spaceKLIP.imagetools:WARNING]  72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411023001_06101_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 0.00 mas


[spaceKLIP.imagetools:WARNING]   --> The following frames might not be properly aligned: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
[spaceKLIP.imagetools:WARNING]  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
[spaceKLIP.imagetools:WARNING]  48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
[spaceKLIP.imagetools:WARNING]  72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411027001_03105_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 0.00 mas


[spaceKLIP.imagetools:WARNING]   --> The following frames might not be properly aligned: [0 1 2 3 4 5 6 7 8 9]


[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411027001_03105_00002_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 21.21 mas
[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411027001_03105_00003_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 21.21 mas
[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411027001_03105_00004_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 21.21 mas
[spaceKLIP.imagetools:INFO]   --> Align frames: jw01411027001_03105_00005_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 21.21 mas
[spaceKLIP.imagetools:INFO]   --> Align frames: jw04454001001_03106_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   --> Align frames: median required shift = 7.28 mas
[spaceKLIP.imagetools:INFO]   --> Align frames: jw04454002001_03106_00001_nrca2_calints.fits
[spaceKLIP.imagetools:INFO]   -->

In [ ]:
Database.obs['JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_FULL']

In [ ]:
# if not aligned and coadd:
ImageTools.coadd_frames()

In [42]:
import platform
if platform.processor() == 'arm':
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["OMP_NUM_THREADS"] = "1" 
# once other image reduction steps are done, run this so that pyklip.parallelized doesn't kill your apple chip

pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [4],
                               # 'movement': [0.5],
                               'subsections': [3],
                                #'numbasis': [1, 2, 5, 8, 10,],
                               'numbasis': [1, 5, 10, 20, 50, 100, 150, 200],#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 35, 50],
                               'algo': 'klip'},
                       subdir='klipsub' #f'klipsub{epoch}'
                      )

[spaceKLIP.pyklippipeline:INFO] --> Concatenation JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R
[spaceKLIP.pyklippipeline:INFO]   --> pyKLIP: mode = ADI+RDI, annuli = 4, subsections = 3
Begin align and scale images for each wavelength
Wavelength 2.098e-06 with index 0 has finished align and scale. Queuing for KLIP
Total number of tasks for KLIP processing is 12


  0%|          | 0/12 [00:00<?, ?it/s]

Closing threadpool
section is too small (0 pixels), skipping...
section is too small (0 pixels), skipping...
section is too small (0 pixels), skipping...
Derotating Images...
Writing Images to directory /Users/rkane/Documents/DATA/JWST/NIRCam/04451/F210M/klipsub
wavelength collapsing reduced data of shape (b, N, wv, y, x):(8, 23, 1, 705, 705)
[spaceKLIP.database:INFO] --> Identified 1 concatenation(s)
[spaceKLIP.database:INFO]   --> Concatenation 1: JWST_NIRCAM_NRCA2_F210M_MASKRND_MASK210R_SUB640A210R
 TYPE   EXP_TYPE DATAMODL ...         KLMODES          BUNIT       BLURFWHM     
------ --------- -------- ... ------------------------ ------ ------------------
PYKLIP NRC_CORON   STAGE3 ... 1,5,10,20,50,100,150,200 MJy/sr 0.9397012604707009


In [ ]:
f'klipsub{epoch}'

In [ ]:
# # select a subset of obs, usually we are only interested in F200W+F444W

# select_obs = [
#               # 'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
#               # 'JWST_NIRCAM_NRCALONG_F356W_MASKRND_MASK335R_SUB320A335R',
#               'JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R',
#               ]

# Database.obs = {k:Database.obs[k] for k in select_obs}
# Database.red = {k:Database.red[k] for k in select_obs}

In [ ]:
# Database.red['JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R']

In [ ]:
Analysis = analysistools.AnalysisTools(Database)

# companions = [[.3173, .0663, 1e-4]]
companions = [[.283, -.074, 1e-4]]

# companion_masks = [[.3173, .0663, 2]]
companion_masks = [[.283, -.074, 2]]

# blob 119.3 and 240.2 pixels
# cen = 320//2
# companions = [[(cen-119.3)*0.063, (240.2-cen)*0.063, 1e-6]]

starfile = data_root+'vizier_votable.vot' #library_subtracted/

mstar_err = 0.0
# {'F200W':0.002773909429007033, 
#              'F356W':0.0302418599954501, 
#              'F444W':0.037934753620330594}

In [ ]:
import astropy

# Read in VOT version
vot_version = astropy.table.Table.read(starfile)

# convert from freq to wl, just for plotting below.
vot_version['wavelength'] = (astropy.constants.c /  vot_version['sed_freq']).to(astropy.units.micron)
plt.clf()
plt.figure()
plt.scatter(vot_version['wavelength'], vot_version['sed_flux'], label='vizier_votable.vot', color='C1')

plt.xlabel("Wavelength [micron]")
plt.ylabel("Flux [Jansky]")
plt.xlim(0,8)
plt.legend(fontsize=9)
plt.show()

In [ ]:
Analysis.raw_contrast(starfile,spectral_type='F0V',companions=companions, subdir='rawcon') #library_subtracted/

In [ ]:
import platform
if platform.processor() == 'arm':
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["OMP_NUM_THREADS"] = "1" 

In [ ]:
inj_seps = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.5, 3.0]
#Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon')
Analysis.calibrate_contrast(companions=companion_masks,
                            injection_seps=inj_seps,
                            plot_xlim=(0,3),
                            subdir='calcon'#f'calcon{epoch}_noiwa_studentt'
                           )

In [ ]:
import platform
if platform.processor() == 'arm':
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["OMP_NUM_THREADS"] = "1" 
# once other image reduction steps are done, run this so that pyklip.parallelized doesn't kill your apple chip

In [ ]:
Analysis.extract_companions(companions, 
                            starfile, 
                            mstar_err, 
                            klmode=10,
                            spectral_type='F0V', 
                            fitmethod='mcmc',
                            fitkernel='matern32',
                            subdir='companions'
#                             # subdir='companions_e1_nestedfix'
#                             subdir='companions_e3_testiwaoff'
                            )

In [ ]:
crash

In [ ]:
# Set the input and output directories and grab the input FITS files.
reduced = True
cleanalign = True
aligned = True

pad = False
coadd = False
crop = False

if aligned and coadd:
    input_dir = './spaceklip/coadded/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    # input_dir = './spaceklip/aligned/'
    input_dir = './spaceklip/aligned_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign:
    input_dir = './spaceklip/nanreplaced/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = './spaceklip/stage2/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    # input_dir = './04558/'
    # input_dir = './uncal_e1/'
    # input_dir = './uncal_e2/'
    input_dir = './uncal_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

output_dir = './spaceklip/'

In [ ]:
# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

In [ ]:
select_obs = [
              # 'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
              'JWST_NIRCAM_NRCALONG_F356W_MASKRND_MASK335R_SUB320A335R',
              'JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

In [ ]:
ImageTools = imagetools.ImageTools(Database)

In [ ]:
pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [4],
                               # 'movement': [0.5],
                               'subsections': [3],
                               'numbasis': [1, 2, 3, 4, 5, 10, 20, 50, 100],
                               'algo': 'klip'},
                       subdir='klipsub_e3')

In [ ]:
Analysis = analysistools.AnalysisTools(Database)

# companions = [[.3173, .0663, 1e-4]]
companions = [[.3147, .0531, 1e-4]]

# blob 119.3 and 240.2 pixels
# cen = 320//2
# companions = [[(cen-119.3)*0.063, (240.2-cen)*0.063, 1e-6]]

starfile = './AFLepA.vot'

mstar_err = 0.0
# {'F200W':0.002773909429007033, 
#              'F356W':0.0302418599954501, 
#              'F444W':0.037934753620330594}

In [ ]:
# Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon_e3')

In [ ]:
# inj_seps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.5, 3.0]
# Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon')
# Analysis.calibrate_contrast(
#                             companions=companions,
#                             injection_seps=inj_seps,
#                             plot_xlim=(0,3),
#                             subdir='calcon_e3'
#                            )

In [ ]:
Analysis.extract_companions(companions, 
                            starfile, 
                            mstar_err, 
                            klmode=100,
                            spectral_type='F8V', 
                            fitmethod='nested',
                            fitkernel='diag',
                            subdir='companions'
                           )

# F200W why do you haate me

In [ ]:
# Set the input and output directories and grab the input FITS files.
reduced = True
cleanalign = True
aligned = True

pad = False
coadd = False
crop = False

if aligned and coadd:
    input_dir = './spaceklip/coadded/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    # input_dir = './spaceklip/aligned/'
    input_dir = './spaceklip/aligned_e1/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign:
    input_dir = './spaceklip/nanreplaced/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = './spaceklip/stage2/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    # input_dir = './04558/'
    # input_dir = './uncal_e1/'
    # input_dir = './uncal_e2/'
    input_dir = './uncal_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

output_dir = './spaceklip/'

# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

select_obs = [
              'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

ImageTools = imagetools.ImageTools(Database)

pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [4],
                               # 'movement': [0.5],
                               'subsections': [3],
                               'numbasis': [1, 2, 3, 4, 5, 10, 20, 50, 100],
                               'algo': 'klip'},
                       subdir='klipsub_e1')

Analysis = analysistools.AnalysisTools(Database)

companions = [[.3173, .0663, 1e-4]]
# companions = [[.3147, .0531, 1e-4]]

# blob 119.3 and 240.2 pixels
# cen = 320//2
# companions = [[(cen-119.3)*0.063, (240.2-cen)*0.063, 1e-6]]

starfile = './AFLepA.vot'

mstar_err = 0.0
# {'F200W':0.002773909429007033, 
#              'F356W':0.0302418599954501, 
#              'F444W':0.037934753620330594}

# Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon_e1')

# inj_seps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.5, 3.0]
# Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon')
# Analysis.calibrate_contrast(
#                             companions=companions,
#                             injection_seps=inj_seps,
#                             plot_xlim=(0,3),
#                             subdir='calcon_e1'
#                            )

Analysis.extract_companions(companions, 
                            starfile, 
                            mstar_err, 
                            klmode=100,
                            spectral_type='F8V', 
                            fitmethod='nested',
                            fitkernel='diag',
                            subdir='companions_e1'
                           )

In [ ]:
# Set the input and output directories and grab the input FITS files.
reduced = True
cleanalign = True
aligned = True

pad = False
coadd = False
crop = False

if aligned and coadd:
    input_dir = './spaceklip/coadded/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    # input_dir = './spaceklip/aligned/'
    input_dir = './spaceklip/aligned_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleanalign:
    input_dir = './spaceklip/nanreplaced/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = './spaceklip/stage2/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    # input_dir = './04558/'
    # input_dir = './uncal_e1/'
    # input_dir = './uncal_e2/'
    input_dir = './uncal_e3/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

output_dir = './spaceklip/'

# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

select_obs = [
              'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

ImageTools = imagetools.ImageTools(Database)

pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [4],
                               # 'movement': [0.5],
                               'subsections': [3],
                               'numbasis': [1, 2, 3, 4, 5, 10, 20, 50, 100],
                               'algo': 'klip'},
                       subdir='klipsub_e3')

Analysis = analysistools.AnalysisTools(Database)

# companions = [[.3173, .0663, 1e-4]]
companions = [[.3147, .0531, 1e-4]]

# blob 119.3 and 240.2 pixels
# cen = 320//2
# companions = [[(cen-119.3)*0.063, (240.2-cen)*0.063, 1e-6]]

starfile = './AFLepA.vot'

mstar_err = 0.0
# {'F200W':0.002773909429007033, 
#              'F356W':0.0302418599954501, 
#              'F444W':0.037934753620330594}

Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon_e3')

inj_seps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.5, 3.0]
Analysis.raw_contrast(starfile,spectral_type='F8V',companions=companions, plot_xlim=(0,3), subdir='rawcon')
Analysis.calibrate_contrast(
                            companions=companions,
                            injection_seps=inj_seps,
                            plot_xlim=(0,3),
                            subdir='calcon_e3'
                           )

# Analysis.extract_companions(companions, 
#                             starfile, 
#                             mstar_err, 
#                             klmode=100,
#                             spectral_type='F8V', 
#                             fitmethod='mcmc',
#                             fitkernel='diag',
#                             subdir='companions_e3'
#                            )

In [ ]:
Analysis.raw_contrast(starfile=data_root+'vizier_votable.vot',
                      spectral_type='F8V',
                      companions=[[.3147, .0531, 1e-4]], 
                      plot_xlim=(0,3), 
                      subdir='rawcon')

In [ ]:
Analysis.calibrate_contrast(
                            companions=[[.3147, .0531, 1e-4]],
                            injection_seps=inj_seps,
                            plot_xlim=(0,3),
                            subdir='calcon'
                           )

In [ ]:
from spaceKLIP.mast import query_coron_datasets
a=query_coron_datasets('NIRCam','F200W','MASKA335R',ignore_ta=True,ignore_cal=True,level='uncal')
a.sort('program')
a[72:]

In [ ]:
ff = sorted(['DATA/JWST/NIRCam/04558_1/F200W/uncal/' + f for f in os.listdir('DATA/JWST/NIRCam/04558_1/F200W/uncal/') if f.endswith('.fits')])

db = database.Database(output_dir='DATA/JWST/NIRCam/04558_1/F200W/')
db.read_jwst_s012_data(datapaths=ff,
                             psflibpaths=None,
                             bgpaths=None)

In [ ]:
Database.obs['JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R'][0:13]